In [ ]:
#Libraries
import numpy as np
import pandas as pd 
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import RandomizedSearchCV, KFold, cross_validate
from sklearn.metrics import f1_score, make_scorer, precision_score, recall_score, average_precision_score, roc_auc_score
import joblib
import os
os.chdir(r'C:/Users/Marc/Desktop/TFG/R Files/Anomaly Detection for SPC')
from Src.FeatureExt.ML_AD_Preprocessing import *

#Pre-processing
#Import data
TS1 = pd.read_csv("Data\TimeSeries1.csv", index_col = 0)

#Standardize data based on first 20 points of chart, which is supposed in control       
X_train = stdvector(TS1)

X_train = sw_dataset_3(X_train, 20)
X_train = np.transpose(X_train)
X_train = pd.DataFrame(X_train,\
     columns = ["last_value", "mean20", "sigma20","mean5", "sigma5", "find_if", "kurtosis","dir_change", 'wavg', 'slope', 'meancross', 'rdist', 'brange'])

#Split dataset for training
split= int(4/5 * X_train.shape[0])

X_train = X_train.iloc[0:split, :]

#Structuring labels for hyperparameter tuning
TS1_Class = pd.read_csv("Data\TimeSeries1_Classification.csv", index_col = 0)
TS1_Class = TS1_Class.iloc[0:1280, 19:60]
X_labels = np.array(TS1_Class).reshape(TS1_Class.size)
X_labels[X_labels == 1] = -1
X_labels[X_labels == 0] = 1


#Define Model
random_state = np.random.RandomState(42)
iforest = IsolationForest()

#Cross Validation
cv = KFold(n_splits = 5, shuffle=True,  random_state = 123)
sc = make_scorer(f1_score)

iforestcv=cross_validate(iforest, X_train, X_labels, cv = cv, scoring=sc)
print(iforestcv)

In [ ]:
print(np.mean(iforestcv['test_score']),
np.std(iforestcv['test_score']))

In [22]:
#Libraries
import numpy as np
import pandas as pd 
from sklearnex import patch_sklearn 
patch_sklearn()
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV, KFold 
from sklearn.metrics import f1_score, make_scorer, accuracy_score, precision_score, recall_score, average_precision_score
import joblib
os.chdir(r'C:/Users/Marc/Desktop/TFG/R Files/Anomaly Detection for SPC')
from Src.FeatureExt.ML_AD_Preprocessing import *

#Pre-processing
#Import data
TS1 = pd.read_csv("Data\TimeSeries1.csv", index_col = 0)

#Standardize data based on first 20 points of chart, which is supposed in control       
X_train = stdvector(TS1)  

#Creating sliding windows for each TS and get features
X_train = sw_dataset_3(X_train, 20)
X_train = np.transpose(X_train)
scaler = StandardScaler()
scaler = scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_train = pd.DataFrame(X_train,\
     columns = ["last_value", "mean20", "sigma20","mean5", "sigma5", "find_if", "kurtosis","dir_change", 'wavg', 'slope', 'meancross', 'rdist', 'brange'])

#Split dataset for training
split= int(4/5 * X_train.shape[0])

X_train = X_train.iloc[0:split, :]

#Training supervised SVM
#Structuring labels
TS1_Class = pd.read_csv("Data\TimeSeries1_Classification.csv", index_col = 0)
TS1_Class = TS1_Class.iloc[0:1280, 19:60]
X_labels = np.array(TS1_Class).reshape(TS1_Class.size)

SVM1 = svm.SVC(kernel="linear")
SVM2 = svm.SVC()

#Cross validation and hyperparameter tuning
cv = KFold(n_splits = 5, shuffle=True,  random_state = 123)
sc = make_scorer(f1_score)
SVMcv1=cross_validate(SVM1, X_train, X_labels, cv = cv, scoring=sc)
SVMcv2=cross_validate(SVM2, X_train, X_labels, cv = cv, scoring=sc)
print(np.mean(SVMcv1['test_score']),
np.std(SVMcv1['test_score']))
print(np.mean(SVMcv2['test_score']),
np.std(SVMcv2['test_score']))

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


0.5666211429141634 0.0034163356308370614
0.7931958927887024 0.0038542172289611136
